## ThinkDSP

This notebook contains code examples from Chapter 6: Discrete Cosine Transform

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
%matplotlib inline

import thinkdsp
import thinkplot

import numpy as np

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

PI2 = np.pi * 2

### Synthesis

The simplest way to synthesize a mixture of sinusoids is to evaluate the sinusoids and add them up.

In [ ]:
def synthesize1(amps, fs, ts):
    components = [thinkdsp.CosSignal(freq, amp)
                  for amp, freq in zip(amps, fs)]
    signal = thinkdsp.SumSignal(*components)

    ys = signal.evaluate(ts)
    return ys

Here's an example that's a mixture of 4 components.

In [ ]:
amps = np.array([0.6, 0.25, 0.1, 0.05])
fs = [100, 200, 300, 400]
framerate = 11025

ts = np.linspace(0, 1, framerate, endpoint=False)
ys = synthesize1(amps, fs, ts)
wave = thinkdsp.Wave(ys, ts, framerate)
wave.apodize()
wave.make_audio()

We can express the same process using matrix multiplication.

In [ ]:
def synthesize2(amps, fs, ts):
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    ys = np.dot(M, amps)
    return ys

And it should sound the same.

In [ ]:
ys = synthesize2(amps, fs, ts)
wave = thinkdsp.Wave(ys, framerate)
wave.apodize()
wave.make_audio()

And we can confirm that the differences are small.

In [ ]:
ys1 = synthesize1(amps, fs, ts)
ys2 = synthesize2(amps, fs, ts)
max(abs(ys1 - ys2))

### Analysis

The simplest way to analyze a signal---that is, find the amplitude for each component---is to create the same matrix we used for synthesis and then solve the system of linear equations.

In [ ]:
def analyze1(ys, fs, ts):
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    amps = np.linalg.solve(M, ys)
    return amps

Using the first 4 values from the wave array, we can recover the amplitudes.

In [ ]:
n = len(fs)
amps2 = analyze1(ys[:n], fs, ts[:n])
amps2

What we have so far is a simple version of a discrete cosine tranform (DCT), but it is not an efficient implementation because the matrix we get is not orthogonal.

In [ ]:
# suppress scientific notation for small numbers
np.set_printoptions(precision=3, suppress=True)

In [ ]:
def test1():
    amps = np.array([0.6, 0.25, 0.1, 0.05])
    N = 4.0
    time_unit = 0.001
    ts = np.arange(N) / N * time_unit
    max_freq = N / time_unit / 2
    fs = np.arange(N) / N * max_freq
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    return M

M = test1()
M

To check whether a matrix is orthogonal, we can compute $M^T M$, which should be the identity matrix:

In [ ]:
M.transpose().dot(M)

But it's not.

Solving a linear system with a general matrix (that is, one that does not have nice properties like orthogonality) takes time proportional to $N^3$.  With an orthogonal matrix, we can get that down to $N^2$.  Here's how:

In [ ]:
def test2():
    amps = np.array([0.6, 0.25, 0.1, 0.05])
    N = 4.0
    ts = (0.5 + np.arange(N)) / N
    fs = (0.5 + np.arange(N)) / 2
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    return M
    
M = test2()
M

Now $M^T M$ is $2I$ (approximately), so M is orthogonal except for a factor of two.

In [ ]:
M.transpose().dot(M)

And that means we can solve the analysis problem using matrix multiplication.

In [ ]:
def analyze2(ys, fs, ts):
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    amps = M.dot(ys) / 2
    return amps

It works:

In [ ]:
n = len(fs)
amps2 = analyze1(ys[:n], fs, ts[:n])
amps2

### DCT

What we've implemented is DCT-IV, which is one of several versions of DCT using orthogonal matrices.

In [ ]:
def dct_iv(ys):
    N = len(ys)
    ts = (0.5 + np.arange(N)) / N
    fs = (0.5 + np.arange(N)) / 2
    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    amps = np.dot(M, ys) / 2
    return amps

We can check that it works:

In [ ]:
amps = np.array([0.6, 0.25, 0.1, 0.05])
N = 4.0
ts = (0.5 + np.arange(N)) / N
fs = (0.5 + np.arange(N)) / 2
ys = synthesize2(amps, fs, ts)

amps2 = dct_iv(ys)
print(max(abs(amps - amps2)))

DCT and inverse DCT are the same thing except for a factor of 2.

In [ ]:
def inverse_dct_iv(amps):
    return dct_iv(amps) * 2

And it works:

In [ ]:
amps = [0.6, 0.25, 0.1, 0.05]
ys = inverse_dct_iv(amps)
amps2 = dct_iv(ys)
print(max(abs(amps - amps2)))

### thinkdsp.Dct

`thinkdsp` provides a `Dct` class that encapsulates the DCT in the same way the Spectrum class encapsulates the FFT.

In [ ]:
signal = thinkdsp.TriangleSignal(freq=400)
wave = signal.make_wave(duration=1.0, framerate=10000)
wave.make_audio()

To make a Dct object, you can invoke `make_dct` on a Wave.

In [ ]:
dct = wave.make_dct()
dct.plot()
thinkplot.config(xlabel='Frequency (Hz)', ylabel='DCT')

Dct provides `make_wave`, which performs the inverse DCT.

In [ ]:
wave2 = dct.make_wave()

The result is very close to the wave we started with.

In [ ]:
max(abs(wave.ys-wave2.ys))

Negating the signal changes the sign of the DCT.

In [ ]:
signal = thinkdsp.TriangleSignal(freq=400, offset=0)
wave = signal.make_wave(duration=1.0, framerate=10000)
wave.ys *= -1
wave.make_dct().plot()

Adding phase offset $\phi=\pi$ has the same effect.

In [ ]:
signal = thinkdsp.TriangleSignal(freq=400, offset=np.pi)
wave = signal.make_wave(duration=1.0, framerate=10000)
wave.make_dct().plot()

# Excercises 

## 6.1

In this chapter I claim that analyze1 takes time proportional
to n^3 and analyze2 takes time proportional to n^2. To see if that’s true, run
them on a range of input sizes and time them. In Jupyter, you can use the
“magic command” %timeit.

If you plot run time versus input size on a log-log scale, you should get a
straight line with slope 3 for analyze1 and slope 2 for analyze2.
You also might want to test dct_iv and scipy.fftpack.dct.

## 6.2

One of the major applications of the DCT is compression for both sound and images. In its simplest form, DCT-based compression works like this:

1. Break a long signal into segments.

2. Compute the DCT of each segment.

3. Identify frequency components with amplitudes so low they are inaudible, and remove them. Store only the frequencies and amplitudes that remain.

4. To play back the signal, load the frequencies and amplitudes for each segment and apply the inverse DCT.

Implement a version of this algorithm and apply it to a recording of music
or speech. How many components can you eliminate before the difference
is perceptible?

In order to make this method practical, you need some way to store a sparse
array; that is, an array where most of the elements are zero. NumPy provides several implementations of sparse arrays, which you can read about
at http://docs.scipy.org/doc/scipy/reference/sparse.html.

In [ ]:
wave = thinkdsp.read_wave('100475__iluppai__saxophone-weep.wav')
wave.make_audio()

In [ ]:
# short segment
segment = wave.segment(start=1.2, duration=0.5)
segment.normalize()
segment.make_audio()

In [ ]:
# DCT of that segment
seg_dct = segment.make_dct()
seg_dct.plot(high=4000)
thinkplot.config(xlabel='Frequency (Hz)', ylabel='DCT')

There are only a few harmonics with substantial amplitude, and many entries near zero.

The following function takes a DCT and sets elements below `thresh` to 0.

In [ ]:
""" Take a DCT and set elements below threshold to 0"""
def compress(dct, thresh=1):
    count = 0
    for i, amp in enumerate(dct.amps):
        if abs(amp) < thresh:
            dct.hs[i] = 0
            count += 1
            
    n = len(dct.amps)
    #print("Count: ", count)
    #print("Total: ", n)
    #print("Eliminated: ", 100 * count / n,"%")
    print(count, n, 100 * count / n, sep='\t')

In [ ]:
seg_dct = segment.make_dct()
compress(seg_dct, thresh=10)
seg_dct.plot(high=4000)

In [ ]:
seg2 = seg_dct.make_wave()
seg2.make_audio()

To compress a longer segment, we can make a DCT spectrogram.

The following function is similar to `wave.make_spectrogram` except that it uses the DCT.

In [ ]:
def make_dct_spectrogram(wave, seg_length):
    """Computes the DCT spectrogram of the wave.

    seg_length: number of samples in each segment

    returns: Spectrogram
    """
    
    window = np.hamming(seg_length) # Create a Hamming window of 'seg_length' length
    i, j = 0, seg_length            # Indexes?
    step = seg_length / 2           # Step size
    
    # map from time to Spectrum
    spec_map = {}

    # Loop while signal is not over
    while j < len(wave.ys):
        segment = wave.slice(i, j)   # Slice from i to j
        segment.window(window)       # Apply a window to it
        
        # the nominal time for this segment is the midpoint
        t = (segment.start + segment.end) / 2
        spec_map[t] = segment.make_dct()
        
        # increase our indexes
        i += step   
        j += step
    
    return thinkdsp.Spectrogram(spec_map, seg_length)

Now we can make a DCT spectrogram and apply `compress` to each segment:

In [ ]:
spectro = make_dct_spectrogram(wave, seg_length=1024)
for t, dct in sorted(spectro.spec_map.items()):
    compress(dct, thresh=0.2)

## 6.3

In the repository for this book you will find a Jupyter notebook
called phase.ipynb that explores the effect of phase on sound perception.
Read through this notebook and run the examples. Choose another segment of sound and run the same experiments. Can you find any general
relationships between the phase structure of a sound and how we perceive
it?